In [24]:
cd /home/laampt/Downloads/BIGLAB/OUT

/home/laampt/Downloads/BIGLAB/OUT


In [25]:
import numpy as np
from gensim.models import word2vec
import gensim
from big_corpus import *

In [26]:
def train(output):
    print 'Training ...'
    model = word2vec.Word2Vec(BigCorpusDirectory('data/play'), workers=num_workers,
                              size=num_features, min_count=min_word_count,
                              window=context, sample=downsampling)

    # If you don't plan to train the model any further, calling
    # init_sims will make the model much more memory-efficient.
    model.init_sims()

    # It can be helpful to create a meaningful model name and
    # save the model for later use. You can load it later using Word2Vec.load()
    model.save(output, ignore=[])

    print model.similarity('queen', 'lady')

In [27]:
# Set values for various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40  # Minimum word count
num_workers = 4  # Number of threads to run in parallel
context = 10  # Context window size
downsampling = 1e-3  # Downsample setting for frequent words
model_name1 = "data/model/lf_300features_40minwords_10context"
model_name2 = "data/model/lf_300features_40minwords_10context_collective"

In [28]:
%%time
train(model_name1)

Training ...
-0.073881448766
CPU times: user 400 ms, sys: 96 ms, total: 496 ms
Wall time: 442 ms


In [29]:
m1 = word2vec.Word2Vec.load("data/model/lf_300features_40minwords_10context")

In [30]:
print m1

Word2Vec(vocab=148, size=300, alpha=0.025)


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
cosine_similarity(m1['queen'], m1['lady'])

array([[-0.07388144]], dtype=float32)

In [39]:
def parallel_train(output):
    def gradient(model, sentences):  # executes on workers
        syn0, syn1 = model.syn0.copy(), model.syn1.copy()  # previous weights
        model.train(sentences)
        return {'syn0': model.syn0 - syn0, 'syn1': model.syn1 - syn1}

    def descent(model, update):  # executes on master
        model.syn0 += update['syn0']
        model.syn1 += update['syn1']

#     root = word2vec.Word2Vec()
#     root.build_vocab(
#         BigCorpusDirectory('data/play')
#     )
#     root.save(output)

    input_files = [
        'data/play/hamlet.txt',
        'data/play/juliet.txt'
    ]

    collective = word2vec.Word2Vec(size=num_features, min_count=min_word_count,
                                   window=context, sample=downsampling)
#     collective.reset_from(root)
    collective.build_vocab(BigCorpusDirectory('data/play'))

    for input_file in input_files:
        individual = word2vec.Word2Vec(size=num_features, min_count=min_word_count,
                                       window=context, sample=downsampling)
        individual.reset_from(collective)

        descent(
            collective,
            gradient(
                individual,
                BigCorpusFile(input_file)
            )
        )
    ## LAMP
    collective.init_sims()
    collective.save(output, ignore=[])
        # individual.train(BigCorpusFile(input_file))
        #
        # collective.syn0 += individual.syn0
        # collective.syn1 += individual.syn1

    print collective.similarity('queen', 'lady')

In [40]:
parallel_train(model_name2)

-0.0658330822267


In [41]:
m2 = word2vec.Word2Vec.load(model_name2)

In [42]:
print m2

Word2Vec(vocab=148, size=300, alpha=0.025)


In [43]:
cosine_similarity(m2['queen'], m2['lady'])

array([[-0.06583308]], dtype=float32)

## Multi epoches on M1, M2

In [44]:
%%time
for i in range(10):
    print "EPO: {:02d}".format(i)
    train(model_name1)

EPO: 00
Training ...
-0.0441904162324
EPO: 01
Training ...
-0.0583321660353
EPO: 02
Training ...
-0.0810477338953
EPO: 03
Training ...
-0.0764315344962
EPO: 04
Training ...
-0.0573930044389
EPO: 05
Training ...
-0.0601818866959
EPO: 06
Training ...
-0.0938245915406
EPO: 07
Training ...
-0.0621270623343
EPO: 08
Training ...
-0.0555930583939
EPO: 09
Training ...
-0.0583459843475
CPU times: user 3.82 s, sys: 3.18 s, total: 7 s
Wall time: 4.42 s


In [45]:
%%time
for i in range(10):
    print "EPO: {:02d}".format(i)
    parallel_train(model_name2)

EPO: 00
-0.0658330822267


EPO: 01
-0.0658330822267


EPO: 02
-0.0658330822267


EPO: 03
-0.0658330822267


EPO: 04
-0.0658330822267


EPO: 05
-0.0658330822267


EPO: 06
-0.0658330822267


EPO: 07
-0.0658330822267


EPO: 08
-0.0658330822267


EPO: 09
-0.0658330822267
CPU times: user 1.22 s, sys: 0 ns, total: 1.22 s
Wall time: 1.11 s


In [47]:
m2.train([["I", "love", "shakespeare"]])

0